# AI cập nhật dữ liệu mới nhất cho bản tin 8h30

In [8]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import get_and_crawl_data
import gemini_model
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(get_and_crawl_data)
importlib.reload(gemini_model)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from get_and_crawl_data import *
from gemini_model import *
from plotly_and_upload import *

### Phần dữ liệu lịch sử các ticker

#### Dữ liệu từ Mongo và AlphaVantage

- Lấy dữ liệu từ Mongo

In [2]:
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1,'volume': 1}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", find_query={"type": "spot"}, projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

- Lấy dữ liệu từ AlphaVantage

In [3]:
# Lấy dữ liệu các tỷ giá cần thiết từ Alpha Vantage
fx_pairs = [
    ('USD', 'VND', 'USD_VND'),
    ('USD', 'SEK', 'USD_SEK')
]

data_frames = []
for from_curr, to_curr, col_name in fx_pairs:
    df = get_data_from_av(from_curr, to_curr, col_name)
    if df is not None:
        data_frames.append(df)

av_ticker_df = pd.concat(data_frames, axis=1)
av_ticker_df = av_ticker_df.fillna(method='ffill').dropna()

#### Chuẩn bị các dữ liệu cần thiết

- Tính toán DXY từ các cặp tiền

In [4]:
currency_config = {
    'EUR_USD': {'source': 'other_ticker_df', 'weight': -0.576},
    'USD_JPY': {'source': 'other_ticker_df', 'weight': 0.136},
    'GBP_USD': {'source': 'other_ticker_df', 'weight': -0.119},
    'USD_CAD': {'source': 'other_ticker_df', 'weight': 0.091},
    'USD_CHF': {'source': 'other_ticker_df', 'weight': 0.036},
    'USD_SEK': {'source': 'av_ticker_df', 'weight': 0.042}
}

# Tạo DataFrame cho mỗi cặp tiền tệ
currency_data = {}
for pair, config in currency_config.items():
    if config['source'] == 'other_ticker_df':
        df = other_ticker_df[other_ticker_df['ticker'] == pair][['date', 'close']].set_index('date')
        df.columns = [pair]
    else:
        df = av_ticker_df[['USD_SEK']].copy()
        df.columns = [pair]
    
    currency_data[pair] = df

# Kết hợp tất cả dữ liệu tỷ giá
dxy_calculation_df = pd.concat(currency_data.values(), axis=1, sort=True)
dxy_calculation_df = dxy_calculation_df.sort_index().bfill().ffill()

# Tính toán chỉ số DXY theo công thức chuẩn
dxy_base = 50.14348112
dxy_calculation = dxy_base
for pair, config in currency_config.items():
    dxy_calculation *= (dxy_calculation_df[pair] ** config['weight'])

dxy_calculation_df['ticker'] = 'DXY'
dxy_calculation_df['close'] = dxy_calculation
dxy_calculation_df = dxy_calculation_df[['ticker', 'close']].reset_index()

- Chuẩn bị tỉ giá USD/VND

In [5]:
usd_vnd_df = av_ticker_df[['USD_VND']].rename(columns={'USD_VND': 'close'})
usd_vnd_df['ticker'] = 'USD_VND'
usd_vnd_df = usd_vnd_df.reset_index()

- Ghép tất cả vào ticker_dict

In [6]:
ticker_dict = {}

# Các ticker từ other_ticker_df
for ticker in ['XAU_USD', 'BTC_USD', 'ETH_USD', 'SSEC', 'DJI', 'N225', 'CLZ', 'FTSE']:
    temp_df = other_ticker_df[other_ticker_df['ticker'] == ticker]
    temp_df = temp_df[['date','ticker','close']]
    temp_df = temp_df.sort_values('date', ascending=False).iloc[:20]
    ticker_dict[ticker] = temp_df

# Các ticker từ full_index_df
for ticker in ['VNINDEX', 'VN30']:
    temp_df = full_index_df[full_index_df['ticker'] == ticker]
    temp_df = temp_df[['date','ticker','close']]
    temp_df = temp_df.sort_values('date', ascending=False).iloc[:20]
    ticker_dict[ticker] = temp_df

#Thêm dữ liệu DXY và USD_VND
ticker_dict['DXY'] = dxy_calculation_df.sort_values('date', ascending=False).iloc[:20]
ticker_dict['USD_VND'] = usd_vnd_df.sort_values('date', ascending=False).iloc[:20]

# Tạo bảng để lưu vào DB
daily_8h30_data_df = pd.DataFrame()
for ticker, df in ticker_dict.items():
    daily_8h30_data_df = pd.concat([daily_8h30_data_df, df], ignore_index=True)

- Vẽ các biểu đồ và lưu vào R2

In [ ]:
# chart_link_dict = {}
# for ticker, df in ticker_dict.items():
#     png_images = create_ticker_chart(
#         df=df.copy().reset_index(drop=True), 
#         height=300, 
#         width=500, 
#         path='../output/cts_daily_chart/', 
#         png_name=f'{ticker}.png'
#     )
#     chart_link_dict[ticker] = upload_to_r2(png_images, f'{ticker}.png', folder_name="cts_daily_chart")
#     break

### Phần dữ liệu cho tin tức

In [ ]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))
model_list = get_gemini_models()

# Sắp xếp danh sách model theo thứ tự ổn định
fast_model_list = select_fast_models(model_list)
standard_model_list = select_standard_models(model_list)

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [ ]:
vietstock_rss_url_dict = {
    'trong_nuoc': {
        'vi_mo': 'https://vietstock.vn/761/kinh-te/vi-mo.rss',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss',
        'chung_khoan_the_gioi': 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss',
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss',
    }
}

cafef_rss_url_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 'https://cafef.vn/vi-mo-dau-tu.chn',
        'chung_khoan': 'https://cafef.vn/thi-truong-chung-khoan.chn',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://cafef.vn/tai-chinh-quoc-te.chn',
    },
    'doanh_nghiep': {
        'doanh_nghiep': 'https://cafef.vn/doanh-nghiep.chn',
    }
}

vietnambiz_rss_url_dict = {
    'trong_nuoc': {
        'chung_khoan': 'https://vietnambiz.vn/chung-khoan/thi-truong.htm',
        'nha_dat': 'https://vietnambiz.vn/nha-dat/thi-truong.htm',
    },
    'quoc_te': {
        'tai_chinh_chung_khoan': 'https://vietnambiz.vn/quoc-te/tai-chinh-chung-khoan.htm',
    },
    'doanh_nghiep': {
        'ket_qua_kinh_doanh': 'https://vietnambiz.vn/doanh-nghiep/ket-qua-kinh-doanh.htm',
    }
}

In [ ]:
vietstock_count_dict = {
    'trong_nuoc': {
        'vi_mo': 2,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 2,
        'chung_khoan_the_gioi': 2,
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 4,
    }
}

cafef_count_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 2,
        'chung_khoan': 2,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 3,
    },
    'doanh_nghiep': {
        'doanh_nghiep': 3,
    }
}

vietnambiz_count_dict = {
    'trong_nuoc': {
        'chung_khoan': 2,
        'nha_dat': 2,
    },
    'quoc_te': {
        'tai_chinh_chung_khoan': 3
    },
    'doanh_nghiep': {
        'ket_qua_kinh_doanh': 3
    }
}

In [ ]:
full_news_df_dict = {}

# Khởi tạo dictionary với list rỗng cho mỗi news_type
for news_type in ['trong_nuoc', 'quoc_te', 'doanh_nghiep']:
    full_news_df_dict[news_type] = []

# Lấy tin từ VietStock
for news_type, url_dict in vietstock_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietstock_count_dict[news_type][category_name]
        feed = feedparser.parse(url)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed.entries[:max_entries]:
            # Get both content and image URL
            content, image_url = get_article_vietstock(entry['id'])
            
            # Lấy thời gian đăng bài từ RSS feed
            published_time = ""
            if hasattr(entry, 'published') and entry.published:
                published_time = entry.published
            
            full_news_df_dict[news_type].append({
                'source': 'VietStock',
                'title': entry['title'], 
                'content': summary_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)
            
# Lấy tin từ CafeF
for news_type, url_dict in cafef_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = cafef_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_cafef_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_cafef(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_cafef_published_time(entry['id'])
            
            full_news_df_dict[news_type].append({
                'source': 'CafeF',
                'title': entry['title'], 
                'content': summary_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)

# Lấy tin từ Vietnambiz 
for news_type, url_dict in vietnambiz_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietnambiz_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_vietnambiz_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_vietnambiz(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_vietnambiz_published_time(entry['id'])
            
            full_news_df_dict[news_type].append({
                'source': 'Vietnambiz',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)
        print(f"Thành công lấy tin từ Vietnambiz: {category_name} ({url})")

In [ ]:
# Chuyển đổi dictionary thành pandas DataFrame
all_news_list = []
for news_type, articles in full_news_df_dict.items():
    for article in articles:
        # Thêm cột news_type vào mỗi article
        article_with_type = article.copy()
        article_with_type['news_type'] = news_type
        all_news_list.append(article_with_type)

# Tạo DataFrame
daily_8h30_news_df = pd.DataFrame(all_news_list)
daily_8h30_news_df['published_time'] = daily_8h30_news_df['published_time'].apply(convert_published_time)
daily_8h30_news_df = daily_8h30_news_df[['news_type'] + [col for col in daily_8h30_news_df.columns if col != 'news_type']]

# Thêm các cột cần thiết
daily_8h30_news_df['word_count'] = daily_8h30_news_df['content'].str.split().str.len()
daily_8h30_news_df['impact'] = analyze_news_impact(standard_model_dict, daily_8h30_news_df)
daily_8h30_news_df['major_news'] = identify_major_news(standard_model_dict, daily_8h30_news_df)

### Lưu dữ liệu vào MSSQL

In [ ]:
%%capture
daily_8h30_news_df.to_sql('daily_8h30_news', cts_engine, if_exists='replace', index=False)
daily_8h30_data_df.to_sql('daily_8h30_data', cts_engine, if_exists='replace', index=False)

In [ ]:
print("=" * 60)
print("🎉 HOÀN THÀNH CẬP NHẬT DỮ LIỆU BẢN TIN CHIẾN LƯỢC HÀNG NGÀY!")
print("=" * 60)
print("✅ Đã lấy và xử lý dữ liệu từ các nguồn:")
print("   • MongoDB (chỉ số VN-Index, VN30)")
print("   • AlphaVantage (tỷ giá USD/VND, USD/SEK)")
print("   • Other tickers (vàng, Bitcoin, chỉ số quốc tế)")
print()
print("✅ Đã tính toán và chuẩn bị:")
print("   • Chỉ số DXY từ 6 cặp tiền tệ")
print("   • Biểu đồ cho 10 ticker và upload lên R2")
print("   • Dữ liệu 20 ngày gần nhất cho mỗi ticker")
print()
print("✅ Đã thu thập và xử lý tin tức:")
print("   • VietStock RSS (vi mô, tài chính quốc tế, doanh nghiệp)")
print("   • CafeF (kinh tế, ngân hàng, chứng khoán, bất động sản)")
print("   • Phân tích tác động và xác định tin quan trọng")
print()
print("✅ Đã lưu dữ liệu vào MSSQL:")
print("   • Bảng daily_8h30_data: Dữ liệu giá các ticker")
print("   • Bảng daily_8h30_news: Tin tức đã phân tích")
print()
print("🚀 Sẵn sàng cho việc tạo báo cáo chiến lược hàng ngày!")
print("=" * 60)